# Излечение, преобразование, загрузка

## Импортируем необходимые модули

In [42]:
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import timedelta

## Настраиваем подключение к БД

In [43]:
user = "entries_user"
password = "entries_password"
host = "localhost"
port = "5432"
database = "entries_db"

engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")

## intervals_tgt

Основа обработки - итерация по датам (ГГГГ-ММ-ДД) для каждого вхождения full_name, рассмотрение каждой записи, формирование из наиболее близких "Вход" и "Выход" интервала. Рассмотрим обработку аномалий:
1. Ошибки системы - из пар записей с одинаковыми статусами и временной разницей менее минуты выбираем более позднюю, другую отбрасываем.
2. Экстренный вход - если выход был произведён в обход системы, то повторный вход не рассматриваем как действительное начало интервала.
3. Ночные выходы - если последней по времени записью для указанной даты является вход, то ищем самый ранний выход в следующем дне. По условию сотрудник не может пропасть или выйти, минуя систему, и не вернуться, учтённый выход на следующий день помечаем, чтобы избежать его повторного учёта.
4. Перерыв на обед - если для указанной даты несколько пар входов и выходов или пара входов и выходов и вход, то учитываем несколько интервалов для одной даты.
5. Конец учётного периода - если сотрудник остался в ночь, совпадающую с окончанием отчётного периода, то его интервал закрывается датой и временем окончания отчётного периода.

1. Группируем события по full_name
2. Сортируем каждую группу по дате по возрастанию.
3. Фильтруем данные по парам:
    - Убираем дубликаты записей (один и тот же статус, время отличается меньше чем на минуту)
    - Убираем записи "Доступ запрещён", когда ожидался "Вход".
    - 

In [ ]:
df = pd.read_sql("SELECT * FROM entries_src ORDER BY full_name, event_dt", engine)
df['event_dt'] = pd.to_datetime(df['event_dt'])

report_period_end = df['event_dt'].max().normalize() + pd.Timedelta(hours=23, minutes=59, seconds=59)

results = []
used_indexes = set()

for name, group in df.groupby('full_name'):
    sorted_group = group.sort_values(by="event_dt")
    rows = list(sorted_group.itertuples(index=True))

    to_drop = []

    for row_i, row_j in zip(rows, rows[1:]):
        if (row_i.status == row_j.status) and (row_j.event_dt - row_i.event_dt < pd.Timedelta(seconds=60)):
            to_drop.append(row_i.Index)

        if (row_i.status == 'Доступ запрещён') and (row_j.status == 'Вход'):
            to_drop.append(row_i.Index)

    sorted_group.drop(to_drop, inplace=True)





2006-04-17 19:22:50 2006-04-18 09:00:00
2006-04-19 19:30:45 2006-04-20 10:18:41
2006-04-20 19:27:32 2006-04-21 10:21:39
2006-05-11 18:09:40 2006-05-12 09:29:39
2006-06-29 18:08:43 2006-06-30 09:05:14
2006-05-09 18:14:15 2006-05-10 08:27:38
2006-06-06 18:21:03 2006-06-07 08:16:16
2006-06-07 17:18:05 2006-06-08 10:04:15
2006-05-22 19:10:30 2006-05-23 10:00:32
2006-04-05 17:21:15 2006-04-06 08:20:54
2006-05-02 17:20:56 2006-05-03 10:24:37
2006-05-15 19:04:07 2006-05-16 09:03:54
2006-06-23 18:18:12 2006-06-26 09:25:44


## workdays_tgt

Для каждого сотрудника рассматриваем все возможные даты (ГГГГ-ММ-ДД), для каждой даты в качестве начала рабочего дня считается наименьшее значение начала интервала, а в качестве окончания рабочего дня считается значение окончания интервала, соответствующее наибольшему значению начала интервала, соответствующей рассматриваемой даты.

In [45]:
df = pd.read_sql("SELECT * FROM intervals_tgt ORDER BY full_name, enter_dt", engine)
df['enter_dt'] = pd.to_datetime(df['enter_dt'])
df['exit_dt'] = pd.to_datetime(df['exit_dt'])

df['report_dt'] = df['enter_dt'].dt.date

results = []

for name, group in df.groupby('full_name'):
    group = group.sort_values('enter_dt').reset_index(drop=True)
    
    grouped_by_date = group.groupby('report_dt')

    for date, day_group in grouped_by_date:
        enter_dt = day_group['enter_dt'].min()
        exit_dt = day_group[day_group['enter_dt'] == day_group['enter_dt'].max()]['exit_dt'].iloc[0]

        results.append({
            'full_name': name,
            'report_dt': date,
            'enter_dt': enter_dt,
            'exit_dt': exit_dt
        })

workdays_df = pd.DataFrame(results)

workdays_df['id'] = workdays_df.index + 1

workdays_df[['id', 'full_name', 'report_dt', 'enter_dt', 'exit_dt']].to_sql(
    'workdays_tgt',
    engine,
    if_exists='append',
    index=False
)

KeyError: "['full_name', 'report_dt', 'enter_dt', 'exit_dt'] not in index"

## aggregated_info_tgt

In [ ]:
query_aggregated_info = """
INSERT INTO aggregated_info_tgt (
    full_name,
    month,
    workdays_count,
    on_time_count,
    late_0_15,
    late_15_30,
    late_30_60,
    late_60_plus,
    full_day_count,
    short_day_count,
    avg_worktime
)
SELECT
    full_name,
    TO_CHAR(report_dt, 'YYYY-MM') AS month,
    COUNT(*) AS workdays_count,
    COUNT(*) FILTER (WHERE enter_dt::time <= TIME '09:00:00') AS on_time_count,
    COUNT(*) FILTER (
        WHERE enter_dt::time > TIME '09:00:00'
          AND EXTRACT(EPOCH FROM (enter_dt::time - TIME '09:00:00')) / 60 <= 15
    ) AS late_0_15,
    COUNT(*) FILTER (
        WHERE EXTRACT(EPOCH FROM (enter_dt::time - TIME '09:00:00')) / 60 > 15
          AND EXTRACT(EPOCH FROM (enter_dt::time - TIME '09:00:00')) / 60 <= 30
    ) AS late_15_30,
    COUNT(*) FILTER (
        WHERE EXTRACT(EPOCH FROM (enter_dt::time - TIME '09:00:00')) / 60 > 30
          AND EXTRACT(EPOCH FROM (enter_dt::time - TIME '09:00:00')) / 60 <= 60
    ) AS late_30_60,
    COUNT(*) FILTER (
        WHERE EXTRACT(EPOCH FROM (enter_dt::time - TIME '09:00:00')) / 60 > 60
    ) AS late_60_plus,
    COUNT(*) FILTER (
        WHERE EXTRACT(EPOCH FROM (exit_dt - enter_dt)) / 3600 >= 9
    ) AS full_day_count,
    COUNT(*) FILTER (
        WHERE EXTRACT(EPOCH FROM (exit_dt - enter_dt)) / 3600 < 9
    ) AS short_day_count,
    ROUND(AVG(EXTRACT(EPOCH FROM (exit_dt - enter_dt)) / 3600), 2) AS avg_worktime
FROM workdays_tgt
GROUP BY full_name, TO_CHAR(report_dt, 'YYYY-MM')
ORDER BY full_name, TO_CHAR(report_dt, 'YYYY-MM');
"""

with engine.connect() as conn:
    with conn.begin():
        conn.execute(text(query_aggregated_info))

## Эскпортируем полученные таблицы в .csv

In [ ]:
tables = ['entries_src', 'intervals_tgt', 'aggregated_info_tgt']

for table in tables:
    df = pd.read_sql(f"SELECT * FROM {table}", engine)
    csv_filename = f"data/target/{table}.csv"
    df.to_csv(csv_filename, index=False)
    print(f"Таблица {table} сохранена в файл {csv_filename}")

Таблица entries_src сохранена в файл data/target/entries_src.csv
Таблица intervals_tgt сохранена в файл data/target/intervals_tgt.csv
Таблица aggregated_info_tgt сохранена в файл data/target/aggregated_info_tgt.csv
